## Change Score Models with one covariate

### 1. Introduction
The simple Change Score Model provides valuable information about the average change and individual differences in change. However, we need to further explore the capabilities of these models, because the intervention effect was not yet included, so we don't know whether the change was different in the intervention as compared with the control group.

In [1]:
#######################################################
## Specify R environment for rpy2
#import os
#os.environ['R_HOME'] = r'C:/Program Files/R/R-4.4.3'  # Replace with your R path
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects import Formula
import contextlib
# Ipython extension for plotting
%load_ext rpy2.ipython
########################################################

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image, display


depression = pd.read_csv("../MLM_and_CSM/Datasets/depression.csv")
depression.head()


c:\Users\danie\.conda\envs\psy112ER\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


,id,group,pre,post,EM,change
0,1,0,34,76,25,42
1,2,0,87,51,34,-36
2,3,0,68,71,15,3
3,4,0,38,72,41,34
4,5,0,82,47,14,-35


### 2. Modeling

Similar to previous section, we use rpy2 also here. For exercise, please use the next code chunk to specify a model (name it m2) that includes the variable "group" as a predictor of pre-intervention depression scores and the phantom variable "ph_change". That is, you will need to add regression paths from this newly included variable group to the "pre" and "ph_change" variables.

**Hint:** Copy the model definition from above and add the relevant model paths. Then run the necessary code to inspect the output.

In [2]:
# Activate pandas2ri for automatic conversion
pandas2ri.activate()

# Import the lavaan package in R
ro.r('library(lavaan)')

# Import semPlot for plotting SEM paths
ro.r('library(semPlot)')

# Load the dataset into R
ro.globalenv['depression'] = pandas2ri.py2rpy(depression)

# Define the SEM model
ro.r('''
m2 <- '
# Fixing change score loading to 1
ph_change =~ 1*post

# Fixing baseline loading to 1
post ~ 1*pre

# Fixing post-treatment score residual variance to 0
post ~~ 0*post

# Fixing post-treatment score intercept to 0
post ~ 0 * 1 

# Freely estimate phantom variable and baseline means. The default would fix them to 0
ph_change ~ 1 
pre ~ 1

# Include covariance between phantom change variable and baseline
ph_change ~~ pre

# Add regression paths from group to pre and ph_change
pre ~ group
ph_change ~ group
'
''')

# Fit the SEM model
ro.r('fit2 <- sem(m2, data=depression, meanstructure=TRUE)')

# Display the summary of the SEM model
summary = ro.r('summary(fit2, fit.measures=TRUE, standardized=TRUE)')
print(summary)


R[write to console]: This is lavaan 0.6-19
lavaan is FREE software! Please report any bugs.



lavaan 0.6-19 ended normally after 178 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                         7

  Number of observations                           100

Model Test User Model:
                                                      
  Test statistic                                 0.000
  Degrees of freedom                                 0

Model Test Baseline Model:

  Test statistic                                58.020
  Degrees of freedom                                 3
  P-value                                        0.000

User Model versus Baseline Model:

  Comparative Fit Index (CFI)                    1.000
  Tucker-Lewis Index (TLI)                       1.000

Loglikelihood and Information Criteria:

  Loglikelihood user model (H0)               -833.998
  Loglikelihood unrestricted model (H1)       -833.998
                                               

### 3. Interpretation

Given the inclusion of the new predictor (group), we can extend our interpretation of the output: 

* Looking at the regression section of the output, we can see that there is a significant relationship between the "group" and "ph_change". Note that "group" is a binary-coded variable (categorical variable). It must be interpreted as a conventional categorical regression using "dummy coding" with the control group as reference group. Consequently, belonging to the intervention group goes along with a larger change in depression, specifically by **24.780** scale points. Once again: Have we seen this number before? It is the same as the "group1:timepost" interaction effect in the MLM "mu3". Pre-therapy scores do not significantly differ between the groups. This is tested by the significance level of the "pre~group" regression estimate (-1.3, p=0.696). 

* Note that the mean change value is **-1.800**. Look back to the interpretation of the MLM "mu3". 